# 看護師離職率データセット作成（ETL完全版）

このノートブックは、手作業によるデータ作成と同等の品質（正確な数値、適切な単位）を自動化するためのものです。
特に**「沖縄県のデータ異常」「年収の単位変換」「家賃の民営借家指定」**などの重要ロジックが実装されています。

## 前提
- 元データ（CSV/Excel）が `../data/raw/` に配置されていること。
- 出力先は `../data/processed/` です。

In [1]:
import pandas as pd
import re
from pathlib import Path

RAW_DIR = Path("../data/raw")

# ==============================
# prefecture 正規化関数
# ==============================
def normalize_prefecture(x):

    if pd.isna(x):
        return x

    x = str(x)

    # 全角半角スペース削除
    x = re.sub(r"\s+", "", x)

    # 全国など除外
    if x in ["全国", "計", "未回答", "無回答・不明"]:
        return None

    # 北海道はそのまま
    if x == "北海道":
        return x

    # 接尾辞補完
    if x == "東京":
        return "東京都"

    if x == "京都":
        return "京都府"

    if x == "大阪":
        return "大阪府"

    # 都道府県が付いてない場合
    if not re.search(r"[都道府県]$", x):
        x += "県"

    return x


# ==============================
# turnover 読み込み
# ==============================
file_turnover = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"

df_turnover = pd.read_csv(file_turnover, encoding="utf-8-sig")

# prefecture 正規化
df_turnover["prefecture"] = df_turnover["prefecture"].apply(normalize_prefecture)

# 不要行削除
df_turnover = df_turnover.dropna(subset=["prefecture"])

# ==============================
# Data Quality Check
# ==============================
print("▼ 行数:", df_turnover.shape[0])
print("▼ 重複:", df_turnover["prefecture"].duplicated().sum())

display(df_turnover.head())


▼ 行数: 47
▼ 重複: 0


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
0,北海道,11.5,5.9,16.6
1,青森県,8.6,10.7,16.7
2,岩手県,6.8,7.8,19.1
3,宮城県,9.1,7.1,12.4
4,秋田県,7.4,5.0,7.3


In [2]:
# ==========================
# Step 1: Master初期化 + DQチェック
# ==========================

df_master = df_turnover.copy()

print("rows:", df_master.shape[0])
print("dup prefecture:", df_master["prefecture"].duplicated().sum())

rows: 47
dup prefecture: 0


もし合格しなかったら（ここも手順固定）
A) rows が 47 じゃない
まず「どの行が余計/不足か」を見ます。

display(df_master["prefecture"].value_counts().head(20))

B) dup が 0 じゃない
「重複している都道府県名」を特定します。

dup_names = df_master[df_master["prefecture"].duplicated(keep=False)].sort_values("prefecture")
display(dup_names[["prefecture"]])

In [3]:
# ==========================
# Step 2: 欠損チェック（必須列が欠けていないか）
# ==========================

# 離職率ドメインの主要指標をまとめた列グループ
# ・DQチェック
# ・数値型変換
# ・レンジ検証（0〜100）
# などで共通利用する
# ※新しい離職率指標を追加した場合は必ずここに追記する
rate_cols = ["turnover_total", "turnover_new_grad", "turnover_experienced"]

print("missing counts:")
print(df_master[rate_cols].isna().sum())


## もし欠損が出た場合（その時だけ実行）
# ==========================
# Step 2b: 欠損している都道府県を特定（原因追跡の入口）
# ==========================

#for c in rate_cols:
#    miss = df_master[df_master[c].isna()][["prefecture", c]]
#   if len(miss) > 0:
#        print(f"\n--- missing in {c} ---")
#        display(miss)

missing counts:
turnover_total          0
turnover_new_grad       0
turnover_experienced    0
dtype: int64


In [4]:
# ==========================
# Step 3: レンジチェック（離職率が0〜100に収まっているか）
# ==========================

# 0未満の値を確認
print("▼ values < 0")
print((df_master[rate_cols] < 0).sum())

# 100超過の値を確認
print("\n▼ values > 100")
print((df_master[rate_cols] > 100).sum())


#もし0じゃなかった場合
#次のセルで原因を特定します。
# ==========================
# Step 3b: 範囲外の都道府県を特定
# ==========================
#
#for col in rate_cols:
#    bad = df_master[
#        (df_master[col] < 0) | (df_master[col] > 100)
#    ][[col]]
#
#    if len(bad) > 0:
#        print(f"\n--- out of range: {col} ---")
#        display(bad)

▼ values < 0
turnover_total          0
turnover_new_grad       0
turnover_experienced    0
dtype: int64

▼ values > 100
turnover_total          0
turnover_new_grad       0
turnover_experienced    0
dtype: int64


In [5]:
# ==========================
# Step 4: prefecture を主キー（index）に固定（再実行しても壊れない）
# ==========================
# 目的: 都道府県をキーに統一し、以降の join を安全にする（冪等: 何回実行しても同じ状態）

# prefecture が列にある場合だけ index 化（すでに index なら何もしない）
if "prefecture" in df_master.columns:
    df_master = df_master.set_index("prefecture")

# index状態の確認（主キーが正しく設定されているか）
print("index name:", df_master.index.name)
print("index duplicated:", df_master.index.duplicated().sum())
display(df_master.head())


index name: prefecture
index duplicated: 0


,turnover_total,turnover_new_grad,turnover_experienced
prefecture,,,
北海道,11.5,5.9,16.6
青森県,8.6,10.7,16.7
岩手県,6.8,7.8,19.1
宮城県,9.1,7.1,12.4
秋田県,7.4,5.0,7.3


In [6]:
# ==========================
# Step 5: night_shift を読み込んでスキーマ確認（まずは中身を見るだけ）
# ==========================
# 目的: join前に「キー列名」「値列名」「不要行の有無」を確定して事故を防ぐ

# Data Sourceのルートパスを定義
RAW_DIR = Path("../data/raw")

# 物理ファイルパスの解決（File Path Resolution）
file_night = RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"
# ソースデータのインジェスト（BOM対応・DataFrameへのロード）
df_night = pd.read_csv(file_night, encoding="utf-8-sig")

# Raw Dataのプレビューによる構造（Schema）の目視確認
display(df_night.head())

# フィールド名（Column Metadata）の一覧取得
print("columns:", df_night.columns.tolist())

# データセットのボリューム（Record Count）の把握
print("rows:", df_night.shape[0])

# 各カラムのデータ型（Storage Type）の整合性確認
print("\n▼ dtypes:")
print(df_night.dtypes)

,prefecture,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,36.7,7.8,4.6
1,青森県,36.5,7.7,4.8
2,岩手県,11.8,7.5,4.1
3,宮城県,30.2,8.0,4.7
4,秋田県,25.1,7.7,4.3


columns: ['prefecture', 'night_shift_72h_plus', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift']
rows: 47

▼ dtypes:
prefecture                             object
night_shift_72h_plus                  float64
night_shifts_per_month_three_shift    float64
night_shifts_per_month_two_shift      float64
dtype: object


In [7]:
# prefectureキーの表記ズレがないか確認（join事故防止）
print(df_night["prefecture"].unique())

['北海道' '青森県' '岩手県' '宮城県' '秋田県' '山形県' '福島県' '茨城県' '栃木県' '群馬県' '埼玉県' '千葉県'
 '東京都' '神奈川県' '新潟県' '富山県' '石川県' '福井県' '山梨県' '長野県' '岐阜県' '静岡県' '愛知県' '三重県'
 '滋賀県' '京都府' '大阪府' '兵庫県' '奈良県' '和歌山県' '鳥取県' '島根県' '岡山県' '広島県' '山口県' '徳島県'
 '香川県' '愛媛県' '高知県' '福岡県' '佐賀県' '長崎県' '熊本県' '大分県' '宮崎県' '鹿児島県' '沖縄県']


In [8]:
# ==========================
# Step 6-1: join前チェック（列名衝突の確認）
# ==========================
# 目的: 既に同名列があると上書きや混乱が起きるため、事前に確認する

# マージ対象とする特徴量（Feature Columns）のホワイトリスト定義
night_cols = [
    "night_shift_72h_plus",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift",
]

# ターゲット（df_master）とのカラム・コンフリクト（列名衝突）の有無を検証
print("overlap columns:", set(night_cols) & set(df_master.columns))

overlap columns: set()


In [9]:
# ==========================
# Step 6-2: night_shift を join用に整形
# ==========================
# 目的: masterと同じキー構造に揃える

# 結合キー（Join Key）をインデックスへ昇格させ、ルックアップ性能を最適化
df_night_idx = df_night.set_index("prefecture")

# インデックス・メタデータが期待通り「prefecture」であることを確認
print("index name:", df_night_idx.index.name)

# 結合時の「意図しない行の増殖（Cartesian Product）」を防ぐための重複チェック
print("index duplicated:", df_night_idx.index.duplicated().sum())

# 整形後のデータセットのカーディナリティ（行数）を最終確認
print("rows:", df_night_idx.shape[0])


index name: prefecture
index duplicated: 0
rows: 47


In [10]:
# ==========================
# Step 6-3: master に night_shift を left join（推奨テンプレ：冪等・安全）
# ==========================
# 目的: master(47都道府県)を維持したまま、夜勤系の列を最新データで入れ直す

# 統合対象とする特徴量（Feature Columns）の単一定義（SSOT: Single Source of Truth）
night_cols = [
    "night_shift_72h_plus",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift",
]

# 結合キー（Join Key）およびユニーク性制約（Uniqueness Constraint）の事前検証
assert df_master.index.name == "prefecture", "df_master の index が prefecture ではありません"
assert df_night_idx.index.name == "prefecture", "df_night_idx の index が prefecture ではありません"
assert df_night_idx.index.duplicated().sum() == 0, "df_night_idx の prefecture が重複しています"

# 冪等性（Idempotency）確保のため、既存のターゲット列を事前にドロップ
df_master = df_master.drop(columns=night_cols, errors="ignore")
#意味: 「もし既に同じ名前の列があったら、一旦消してまっさらにする」という処理です。
#なぜやる？: これがないと、コードを2回実行したときに「同じ名前の列がもうあるよ！」とエラーで止まってしまいます。
#「何度やり直しても、常に同じ結果になる（冪等性）」ようにしておく。


# 処理前レコード件数（Baseline Record Count）のスナップショット取得
before_rows = df_master.shape[0]

# 左外部結合（Left Outer Join）によるエンティティ保持と属性拡張
df_master = df_master.join(df_night_idx[night_cols], how="left")

# 処理後レコード件数（Post-process Record Count）の取得
after_rows = df_master.shape[0]

# 整合性チェック：結合に伴う予期せぬ行増殖（Cartesian Product）の検知
if before_rows != after_rows:
    raise ValueError(f"CRITICAL: Record count changed from {before_rows} to {after_rows}")

# 実行結果のサマリー出力（Execution Logging）
print("rows before:", before_rows)
print("rows after :", after_rows)

# 結合後のデータ充填率（Fill Rate）の確認と欠損値（Missing Value）の集計
missing_counts = df_master[night_cols].isna().sum()
print("\nmissing counts (night):")
print(missing_counts)

# キー不一致（Key Mismatch）による結合失敗エンティティの特定
missing_pref = df_master[df_master[night_cols].isna().any(axis=1)].index.tolist()
print("\nmissing prefectures:", missing_pref)

# 最終的なデータフレーム・スキーマと値のプロファイリング
display(df_master.head())

rows before: 47
rows after : 47

missing counts (night):
night_shift_72h_plus                  0
night_shifts_per_month_three_shift    0
night_shifts_per_month_two_shift      0
dtype: int64

missing prefectures: []


,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
prefecture,,,,,,
北海道,11.5,5.9,16.6,36.7,7.8,4.6
青森県,8.6,10.7,16.7,36.5,7.7,4.8
岩手県,6.8,7.8,19.1,11.8,7.5,4.1
宮城県,9.1,7.1,12.4,30.2,8.0,4.7
秋田県,7.4,5.0,7.3,25.1,7.7,4.3


In [11]:
# ==========================
# Step 6-4: join後DQ（夜勤列の欠損チェック）
# ==========================
# 目的: joinに失敗した都道府県がないかを確認する

print("missing counts (night):")
print(df_master[night_cols].isna().sum())

missing_pref = df_master[df_master[night_cols].isna().any(axis=1)].index.tolist()
print("\nmissing prefectures:", missing_pref)


missing counts (night):
night_shift_72h_plus                  0
night_shifts_per_month_three_shift    0
night_shifts_per_month_two_shift      0
dtype: int64

missing prefectures: []


In [12]:
# ==========================
# Step 7: 中間成果を書き出し（Checkpoint）
# ==========================
# 目的: 統合済みmasterを保存して、次の統合作業で壊れても戻れるようにする

from pathlib import Path

# アプリケーションの出力先ディレクトリ（Target Layer）のパス定義
OUT_DIR = Path("../data/out")
# ディレクトリの冪等な作成（Recursive Directory Creation）
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 物理ファイル名およびストレージパスの解決
out_file = OUT_DIR / "master_step1_turnover_nightshift.csv"
# シリアライズ処理（BOM付きUTF-8での永続化）
df_master.to_csv(out_file, encoding="utf-8-sig")

# 処理完了のロギング（Completion Logs）
print("saved:", out_file)
# 保存済みデータセットのシェイプ（Cardinality & Schema Width）の最終記録
print("rows:", df_master.shape[0], "cols:", df_master.shape[1])

saved: ../data/out/master_step1_turnover_nightshift.csv
rows: 47 cols: 6


In [13]:
# ==========================
# Step 8: density を読み込んで構造確認
# ==========================
# 目的: join前にキー列名・粒度・型を確認する

# 物理ストレージ上のソースファイル・パスの解決（Path Resolution）
file_density = RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"

# 外部ソースデータからのデータ・インジェスト（Data Ingestion）
df_density = pd.read_csv(file_density, encoding="utf-8-sig")

# サンプルレコードの抽出による物理構造（Raw Structure）の目視確認
display(df_density.head())

# フィールド名（Column Metadata）の定義状況を確認
print("columns:", df_density.columns.tolist())

# データセットのカーディナリティ（データ規模とレコード件数）を把握
print("rows:", df_density.shape[0])

# 型推論（Type Inference）の結果が期待するスキーマと一致するか検証
print("\n▼ dtypes")
print(df_density.dtypes)

,統計名：,都道府県データ 社会生活統計指標,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,表番号：,10201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,表題：,Ａ 人口・世帯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,実施年月：,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,市区町村時点（年月日）：,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


columns: ['統計名：', '都道府県データ 社会生活統計指標', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32']
rows: 60

▼ dtypes
統計名：                object
都道府県データ 社会生活統計指標    object
Unnamed: 2          object
Unnamed: 3          object
Unnamed: 4          object
Unnamed: 5          object
Unnamed: 6          object
Unnamed: 7          object
Unnamed: 8          object
Unnamed: 9          object
Unnamed: 10         object
Unnamed: 11         object
Unnamed: 12         object
Unnamed: 13         object
Unnamed: 14         object
Unnamed: 15         object
Unnamed: 16         object
Unnamed: 17         object


In [14]:
# ==========================
# Step 8-1: 正しいヘッダ行を探索（先頭20行を“生”で見る）
# ==========================
# 目的: メタ情報の下にある「本当の列名行」を見つける

# データソースの物理ファイルパスを定義
file_density = RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"

# 自動パースを無効化し、Raw形式でインジェスト（Header Suppression）
df_density_raw = pd.read_csv(file_density, encoding="utf-8-sig", header=None)

# スキップすべき不要なメタデータ行を特定するための、データ・プロファイリング（目視確認）
display(df_density_raw.head(20))

# インジェストされたデータセットの初期ボリューム（Row/Col count）の把握
print("raw shape:", df_density_raw.shape)

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,統計名：,都道府県データ 社会生活統計指標,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,表番号：,10201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,表題：,Ａ 人口・世帯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,実施年月：,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,市区町村時点（年月日）：,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,***,調査又は集計していないもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-,データが得られないもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,X,数値が秘匿されているもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


raw shape: (61, 33)


In [15]:
# ==========================
# Step 8-2: density を正しいヘッダ行で読み直す
# ==========================

# 物理ストレージ上のソースファイル・パスを再定義
file_density = RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"

# 指定した行数（Offset）をスキップし、特定の行を論理スキーマ（Header）として採用
df_density = pd.read_csv(file_density, encoding="utf-8-sig", header=12)

# ロード済みデータセットの先頭レコードによる整合性確認（Sanity Check）
display(df_density.head())

# パースされたフィールド名（Schema Metadata）の一覧を取得
print("columns:", df_density.columns.tolist())

# インジェストされた有効なレコード件数（Active Record Count）の把握
print("rows:", df_density.shape[0])

,調査年 コード,調査年 補助コード,調査年,地域 コード,地域 補助コード,地域,/Ａ 人口・世帯,#A011000_総人口【万人】,#A0110001_総人口（男）【万人】,#A0110002_総人口（女）【万人】,...,#A03501_15歳未満人口割合【％】,#A03502_15～64歳人口割合【％】,#A03503_65歳以上人口割合【％】,#A05101_人口増減率（（A1101/A1101（-1））-1）【％】,#A05301_転入超過率（日本人移動者）【％】,#A05302_転入率（日本人移動者）【％】,#A05303_転出率（日本人移動者）【％】,#A05307_転入超過率【％】,#A05308_転入率【％】,#A05309_転出率【％】
0,2023100000,NaN,2023年度,0,NaN,全国,NaN,"12,435","6,049","6,386",...,11.4,59.5,29.1,-0.48,-,1.79,1.79,-,2.05,2.05
1,2023100000,NaN,2023年度,1000,NaN,北海道,NaN,509,241,269,...,10.1,56.9,33.0,-0.93,-0.11,0.93,1.04,-0.1,1.05,1.15
2,2023100000,NaN,2023年度,2000,NaN,青森県,NaN,118,56,63,...,10.0,54.8,35.2,-1.66,-0.47,1.29,1.76,-0.48,1.37,1.85
3,2023100000,NaN,2023年度,3000,NaN,岩手県,NaN,116,56,60,...,10.3,54.7,35.0,-1.52,-0.41,1.28,1.69,-0.4,1.39,1.79
4,2023100000,NaN,2023年度,4000,NaN,宮城県,NaN,226,111,116,...,11.1,59.7,29.2,-0.70,-0.04,1.90,1.94,-0.06,2.03,2.09


columns: ['調査年 コード', '調査年 補助コード', '調査年', '地域 コード', '地域 補助コード', '地域', '/Ａ\u3000人口・世帯', '#A011000_総人口【万人】', '#A0110001_総人口（男）【万人】', '#A0110002_総人口（女）【万人】', '#A01101_全国総人口に占める人口割合（A1101/A1101(全国)）【％】', '#A01201_総面積１km2当たり人口密度【人】', '#A01202_可住地面積１km2当たり人口密度【人】', '#A0191002_将来推計人口（2025年）【人】', '#A0191003_将来推計人口（2030年）【人】', '#A0191004_将来推計人口（2035年）【人】', '#A0191005_将来推計人口（2040年）【人】', '#A0191006_将来推計人口（2045年）【人】', '#A0191007_将来推計人口（2050年）【人】', '#A02101_人口性比（総数）（A110101/A110102）【‐】', '#A02102_人口性比（15歳未満人口）(A130101/A130102)【‐】', '#A02103_人口性比（15～64歳人口）(A130201/A130202)【‐】', '#A02104_人口性比（65歳以上人口) (A130301/A130302)【‐】', '#A03501_15歳未満人口割合【％】', '#A03502_15～64歳人口割合【％】', '#A03503_65歳以上人口割合【％】', '#A05101_人口増減率（（A1101/A1101（-1））-1）【％】', '#A05301_転入超過率（日本人移動者）【％】', '#A05302_転入率（日本人移動者）【％】', '#A05303_転出率（日本人移動者）【％】', '#A05307_転入超過率【％】', '#A05308_転入率【％】', '#A05309_転出率【％】']
rows: 48


In [16]:
# ==========================
# Step 8-3: 人口密度列を自動で特定（候補を表示）
# ==========================

# スキーマ内の全フィールドから、キーワードに合致するターゲット列を動的に抽出（Pattern Matching）
density_candidates = [c for c in df_density.columns if "人口密度" in str(c)]

# 自動特定された候補カラムのリストを検証用にロギング（Discovery Results）
print("density_candidates:", density_candidates)

density_candidates: ['#A01201_総面積１km2当たり人口密度【人】', '#A01202_可住地面積１km2当たり人口密度【人】']


In [17]:
# ==========================
# Step 8-4: density（総面積ベース）を整形して master に冪等 join
# ==========================
# 目的: prefectureをキーに、人口密度（総面積1km2あたり）をmasterへ追加する（再実行しても壊れない）

# 特定されたターゲット指標の論理カラム名を物理定義
density_col = "#A01201_総面積１km2当たり人口密度【人】"

# アウトオブスコープ（全国）の除外と、サブセットの抽出（Column Selection）
df_density_clean = df_density.loc[df_density["地域"] != "全国", ["地域", density_col]].copy()

# 下流工程（Downstream）のスキーマ標準に合わせたリネーム（Canonical Mapping）
df_density_clean = df_density_clean.rename(columns={"地域": "prefecture", density_col: "density"})

# データの正規化：型変換の妨げとなるノイズ（カンマ・記号）のクリーニング
df_density_clean["density"] = (
    df_density_clean["density"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .replace({"-": None, "***": None, "X": None})
)
# 強制的な型変換（Coercion）による数値データ型へのキャストと欠損値処理
df_density_clean["density"] = pd.to_numeric(df_density_clean["density"], errors="coerce")

# 結合演算の効率化に向けたインデックス化とユニーク制約の検証
df_density_idx = df_density_clean.set_index("prefecture")
print("dup density index:", df_density_idx.index.duplicated().sum())
print("rows density:", df_density_idx.shape[0])

# 冪等性（Idempotency）の確保：既存属性のクリーンアップ
df_master = df_master.drop(columns=["density"], errors="ignore")

# 結合前後のレコード件数比較（Baseline Comparison）
before_rows = df_master.shape[0]
# 左外部結合（Left Outer Join）によるマスタの属性拡張（Feature Enrichment）
df_master = df_master.join(df_density_idx[["density"]], how="left")
after_rows = df_master.shape[0]

# データ整合性バリデーション：行数変化の有無を確認
if before_rows != after_rows:
    raise ValueError(f"CRITICAL: Record count changed from {before_rows} to {after_rows}")

# パイプライン実行ログの出力
print("rows before:", before_rows)
print("rows after :", after_rows)

# 結合成功率（Coverage Check）の算出と欠損分析
print("\nmissing counts (density):")
print(df_master[["density"]].isna().sum())

# キーのミスマッチ（Unmatched Keys）が発生しているエンティティの特定
missing_pref = df_master[df_master["density"].isna()].index.tolist()
print("missing prefectures:", missing_pref)

# 最終的なデータ構造と分布のプロファイリング
display(df_master.head())

# ==========================
# Step 8-4: 人口密度データの統合（作業メモ）
# ==========================
# 人口密度データの「地域」列を、マスタ側と合わせるために「prefecture」へとリネームした。
# データに混入していた「全国」などの集計行は、分析の邪魔になるため事前に除外してある。
# 数字に含まれるカンマや「-」などの記号をクリーニングし、計算ができるよう数値型へ変換した。
# その際、変換できない不正な値はエラーにせず、安全に欠損値（NaN）として処理している。
# 結合時にデータが重複して増えないよう、都道府県をインデックスに設定して一意性を確保した。
# コードを何度再実行してもエラーにならないよう、既存の列を一度削除してから結合する設計にした。
# マスタにある47都道府県の行を削らないよう、Left Join（左外部結合）で人口密度を統合。
# 結合の前後で行数が変わっていないかを検証し、データが壊れていないことを保証した。
# 最後に、マスタ側の県名と一致せず結合に失敗した箇所がないか、欠損数を数えて確認した。

dup density index: 0
rows density: 47
rows before: 47
rows after : 47

missing counts (density):
density    0
dtype: int64
missing prefectures: []


,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift,density
prefecture,,,,,,,
北海道,11.5,5.9,16.6,36.7,7.8,4.6,64.9
青森県,8.6,10.7,16.7,36.5,7.7,4.8,122.8
岩手県,6.8,7.8,19.1,11.8,7.5,4.1,76.1
宮城県,9.1,7.1,12.4,30.2,8.0,4.7,310.9
秋田県,7.4,5.0,7.3,25.1,7.7,4.3,78.5


In [18]:
# ==========================
# Step 8-5: Checkpoint保存（turnover + night + density）
# ==========================

# 出力先ディレクトリが存在しない場合に備えた、ディレクトリの動的生成
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 中間成果物（Artifact）の物理保存パスの定義
checkpoint_path = OUT_DIR / "master_step2_density.csv"

# インデックスの列復元（Flattening）を行い、ポータビリティの高い形式でCSV出力
df_master.reset_index().to_csv(
    checkpoint_path,
    index=False,
    encoding="utf-8-sig"
)

# 保存完了のロギング（Persistence Confirmation）
print("saved:", checkpoint_path)
# 出力されたデータセットの次元（Cardinality & Schema Width）の最終確認
print("rows:", df_master.shape[0], "cols:", df_master.shape[1])

saved: ../data/out/master_step2_density.csv
rows: 47 cols: 7


In [19]:
# ==========================
# Step 8-6: density → population_density に正式名称統一
# ==========================

# フィールド名の曖昧さを排除し、ドメイン知識に基づいた標準名称へリネーム（Semantic Labeling）
df_master = df_master.rename(columns={
    "density": "population_density"
})

# 修正後のスキーマ（Schema）に定義の齟齬がないか最終確認
print("columns:", df_master.columns.tolist())

# カラム名変更後のデータセットの物理状態をプレビュー（Final Inspection）
display(df_master.head())

columns: ['turnover_total', 'turnover_new_grad', 'turnover_experienced', 'night_shift_72h_plus', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift', 'population_density']


,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift,population_density
prefecture,,,,,,,
北海道,11.5,5.9,16.6,36.7,7.8,4.6,64.9
青森県,8.6,10.7,16.7,36.5,7.7,4.8,122.8
岩手県,6.8,7.8,19.1,11.8,7.5,4.1,76.1
宮城県,9.1,7.1,12.4,30.2,8.0,4.7,310.9
秋田県,7.4,5.0,7.3,25.1,7.7,4.3,78.5
